In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/face-mask-dataset


In [3]:
import os

# Print file/folder list
print(os.listdir(path))

['data']


In [4]:
import zipfile

# Check if zip file exists
zip_path = os.path.join(path, "face-mask-dataset.zip")  # Adjust the file name if necessary
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path)
    print("Dataset has been unzip!")
    print(os.listdir(path))  # Check the content again after unzipping

In [5]:
import cv2
import os

# Suppose the folders containing the images are 'data/with_mask' and 'data/without_mask'
with_mask_dir = os.path.join(path, 'data/with_mask')  # Adjust to actual pathế
without_mask_dir = os.path.join(path, 'data/without_mask')

# Load a sample image
sample_image_path = os.path.join(with_mask_dir, os.listdir(with_mask_dir)[0])
img = cv2.imread(sample_image_path)
print("Sample image size:", img.shape)

# List the number of images in each folder
print("Number of photos wearing masks:", len(os.listdir(with_mask_dir)))
print("Number of photos without masks:", len(os.listdir(without_mask_dir)))

Sample image size: (800, 800, 3)
Number of photos wearing masks: 3725
Number of photos without masks: 3828


In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time

# Thiết lập các tham số
# data_dir = 'dataset'      # đường dẫn đến folder dataset
data_dir = os.path.join(path, 'data')
num_classes = 2           # Có 2 lớp: with_mask và without_mask
batch_size = 32
num_epochs = 10
learning_rate = 0.001
input_size = 128

# Sử dụng GPU nếu có
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Thiết bị sử dụng: {device}")

# Định nghĩa chuyển đổi dữ liệu: resize, crop, normalization, …
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # giá trị chuẩn cho ImageNet
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# Tạo tập dữ liệu với 80% train và 20% validation
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])
dataset_size = len(full_dataset)
indices = list(range(dataset_size))
split = int(0.2 * dataset_size)

# Khuynh hướng trộn dữ liệu
import numpy as np
np.random.shuffle(indices)
train_idx, val_idx = indices[split:], indices[:split]

from torch.utils.data.sampler import SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

train_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=4)
val_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=val_sampler, num_workers=4)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_idx), 'val': len(val_idx)}

# Định nghĩa một kiến trúc CNN đơn giản (hoặc dùng transfer learning nếu muốn)
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            # Lớp 1
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Lớp 2
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Lớp 3
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Tính toán kích thước feature sau khi pooling 3 lần:
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Khởi tạo mô hình và chuyển sang device
model = SimpleCNN(num_classes=num_classes)
model = model.to(device)

# Định nghĩa loss function và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Hàm huấn luyện
def train_model(model, criterion, optimizer, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0
    since = time.time()

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-'*10)

        # Mỗi epoch có 2 giai đoạn: train và validation
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Duyệt qua dữ liệu
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward propagation
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward propagation nếu đang ở giai đoạn train
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Lưu mô hình tốt nhất
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()
    time_elapsed = time.time() - since
    print(f'Huấn luyện hoàn tất trong {time_elapsed//60:.0f} phút {time_elapsed%60:.0f} giây')
    print(f'Accuracy tốt nhất: {best_acc:.4f}')

    # Tải trọng số tốt nhất
    model.load_state_dict(best_model_wts)
    return model

# Huấn luyện mô hình
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)

# Lưu mô hình đã huấn luyện
torch.save(model.state_dict(), '/content/drive/MyDrive/Project/Face Mask Detection/model/face_mask_detector.pth')
print("Mô hình đã được lưu vào face_mask_detector.pth")


Thiết bị sử dụng: cuda:0


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1/10
----------


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.3235 Acc: 0.8636
val Loss: 0.3059 Acc: 0.8662

Epoch 2/10
----------


...

train Loss: 0.1065 Acc: 0.9583
val Loss: 0.1263 Acc: 0.9583

Epoch 9/10
----------


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.1067 Acc: 0.9605
val Loss: 0.2565 Acc: 0.8907

Epoch 10/10
----------


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.0945 Acc: 0.9654
val Loss: 0.1211 Acc: 0.9589

Huấn luyện hoàn tất trong 3 phút 34 giây
Accuracy tốt nhất: 0.9589
Mô hình đã được lưu vào face_mask_detector.pth
